In [ ]:
from google.colab import drive 
drive.mount("/content/drive/")
import pandas as pd
import os 
import re
import json
os.chdir("/content/drive/MyDrive/tesi/chapters/Lexsem validation/")
from zipfile import ZipFile
import zipfile
import statistics
from statistics import mean
import collections
from collections import Counter
from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu

Mounted at /content/drive/


# **SETUP**

In [ ]:
! pip install pingouin 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 19.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7048 sha256=07a1fec346f7e7d17164023509e61e185acda60ddb3fa72ad1f977e7904e5ab4
  Stored in directory: /root/.cache/pip/wheels/04/bb/0d/2d02ec45f29c48d6192476bfb59c5a0e64b605e7212374dd15
Successfully built littleutils


In [ ]:
!pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 30.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895259 sha256=59d88e0782bd23eb2e1fdd99cf3ad2c5ec66d9ef61d5bca6c71fa5b9e6123fd6
  Stored in directory: /root/.cache/pip/wheels/12/1c/3d/46cf06718d63a32ff798a89594b61e7f345ab6b36d909ce033
Successfully built sacremoses


In [ ]:
!pip install sacrebleu
import sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 10.8 MB/s eta 0:00:00


In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 100.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.3 MB/s eta 0:00:00


In [ ]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=7048371ac55d8bf9658047a95309bb252cf9b057b32794345f44fd5203ca5d67
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer
model_ = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
!pip install --upgrade bleu
import bleu
from bleu import list_bleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for bleu: filename=bleu-0.3-py3-none-any.whl size=5799 sha256=1c84d2b1fd239185b789518e7f9b67d91c7a88138e8fc7431ecb8d1f87d928fc
  Stored in directory: /root/.cache/pip/wheels/78/bf/47/ded78149e63219f4666fba6ffc55697b8db6282c87d6370a7e
  Created wheel for efficiency: filename=efficiency-1.1-py3-none-any.whl size=22157 sha256=7fe51f71fd2dfc0c4581a048a26606c86770c74f19436b9a1019ca44d7e780d0
  Stored in directory: /root/.cache/pip/wheels/b1/46/0d/82133b0346045c28acaa24d9e2def1044e92441ffaae95f252
Successfully built bleu efficiency


In [ ]:
from nltk.translate.bleu_score import sentence_bleu

def compute_scar_score(list_):

  bleu = []
  scar = []
  cc = SmoothingFunction()

  for i, l_ in enumerate(list_):

    reference = list_[i][0]

    translation = list_[i][1]

    bleu_score = sentence_bleu(reference, translation, weights=(0.5, 0.5, 0, 0),smoothing_function= cc.method3)
    
    bleu.append(round(bleu_score, 2))

    
  return mean(bleu) , bleu 


In [ ]:
import statistics
from statistics import mean

def compute_similarity(list_):

  similarity = []
  for l_ in list_:
    sentence_embeddings = model_.encode(l_)
    similarity.append(round(util.pytorch_cos_sim(sentence_embeddings[0], sentence_embeddings[1]).item(), 2))


  return mean(similarity), similarity 

# **Data-gathering**

In [ ]:
!pip install datasets
from datasets import load_dataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 16.5 MB/s eta 0:00:00


In [ ]:
# Load the WMT16 dataset
dataset_ = load_dataset('qanastek/EMEA-V3', 'de-it', split=['train']) 

# Access the data splits
train_data = dataset_[0]
# val_data = dataset_[1]
# test_data = dataset_[2]

# Print some statistics
print(f"Number of training examples: {len(train_data)}")
# print(f"Number of validation examples: {len(val_data)}")
# print(f"Number of test examples: {len(test_data)}")

Generating train split: 0 examples [00:00, ? examples/s]

Dataset emea-v3 downloaded and prepared to /root/.cache/huggingface/datasets/qanastek___emea-v3/de-it/3.0.0/e62de864dc5626af0372147e7d402cc77d7be714b3173a9cda62cbf77080dba4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Number of training examples: 381396


In [ ]:
data = dataset_[0][5:45]["translation"]

In [ ]:
bitext = []
original = []
translation = []
for i, x in enumerate(data):
  for y in x.items():
    # print(y)
    bitext.append(y[1])

for i, text in enumerate(bitext):
  if i % 2 == 0:
    translation.append(text.lower())
  else:
    original.append(text.lower())

In [ ]:

pd.DataFrame({"original": translation , "translation": original}).to_csv("de-it.csv")
pd.read_csv("de-it.csv", index_col = 0)

,original,translation
0,wofür wird abilify angewendet?,per che cosa si usa abilify?
1,"b. wirres denken und sprechen, halluzinationen...",abilify è indicato per il trattamento di pazie...
2,es können auch depressive episoden auftreten.,possono inoltre verificarsi episodi di depress...
3,abilify wird zur behandlung mittelschwerer bis...,abilify è impiegato per il trattamento di epis...
4,die injektionslösung wird zur schnellen kontro...,la soluzione iniettabile è utilizzata per il c...
5,wie wird abilify angewendet?,come si usa abilify?
6,bei schizophrenie beträgt die empfohlene anfan...,"in caso di schizofrenia, la dose di partenza r..."
7,die erhaltungsdosis beträgt 15 mg einmal tägli...,la dose di mantenimento è di 15 mg una volta a...
8,bei bipolarer störung beträgt die empfohlene a...,"per il disturbo bipolare, la dose di partenza ..."
9,bei einigen patienten kann auch eine höhere do...,alcuni pazienti possono trarre beneficio da un...


# **TRANSLATION**

In [ ]:
data_ = pd.read_excel('de-it-modified-file.xlsx', index_col = 0)



In [ ]:
data_["original_sentences"]

0                        wofür wird abilify angewendet?
2         es können auch depressive episoden auftreten.
3     abilify wird zur behandlung mittelschwerer bis...
4     die injektionslösung wird zur schnellen kontro...
5                          wie wird abilify angewendet?
6     bei schizophrenie beträgt die empfohlene anfan...
7     die erhaltungsdosis beträgt 15 mg einmal tägli...
8     bei bipolarer störung beträgt die empfohlene a...
9     bei einigen patienten kann auch eine höhere do...
10    zur vorbeugung manischer episoden sollte die g...
11    bei beiden erkrankungen können die lösung zum ...
12    die schmelztabletten werden auf die zunge gele...
13    die übliche dosis beträgt 9,75 mg als einzelne...
14    falls erforderlich, kann eine zweite injektion...
15    abilify kann mit dem essen oder auf nüchternen...
16    diese dosis sollte jedoch bei patienten, die s...
17    bei patienten, die gleichzeitig andere arzneim...
18    abilify wurde nicht bei kindern unter 18 j

In [ ]:
import glob

def german_italian_translator(path):    #TAKES A PATH TO FILE AS ARGUMRNT,  




  dataframe = pd.read_excel(path, index_col = 0)                          

 
                                                   
  original = [sent_ for sent_ in dataframe["original_sentences"] ]





  direct_translator = pipeline("translation", model="Helsinki-NLP/opus-mt-de-it" )



  direct_translation = [direct_translator(text, max_length=400) [0] ['translation_text'] for text in original]

  
  return  original, direct_translation

In [ ]:
path_ = "/content/drive/MyDrive/tesi/chapters/Lexsem validation/de-it-modified-file.xlsx"

original_sentences, direct_translation = german_italian_translator(path_)

In [ ]:
pd.DataFrame({"original":  original_sentences , "direct_translation": direct_translation}).to_csv("de-it-DT.csv")

In [ ]:
dt_translation = pd.read_csv("de-it-DT.csv", index_col = 0)

In [ ]:
h_translation = pd.read_excel("de-it-modified-file.xlsx", index_col = 0)    

In [ ]:
original = [sent_ for sent_ in dt_translation["original"]]

direct_translation = [sent_ for sent_ in dt_translation["direct_translation"]]

reference_translation = [sent_ for sent_ in h_translation["reference_translation"]]



In [ ]:
pd.DataFrame({"original_sentences":  original , "direct_translation": direct_translation, "reference_translation": reference_translation}).to_csv("german_italian_dt_ht.csv")

In [ ]:
pd.read_csv("german_italian_dt_ht.csv", index_col = 0).head()

,original_sentences,direct_translation,reference_translation
0,wofür wird abilify angewendet?,A cosa serve Abilify,per che cosa si usa abilify?
1,es können auch depressive episoden auftreten.,possono verificarsi anche episodi di depressione.,possono inoltre verificarsi episodi di depress...
2,abilify wird zur behandlung mittelschwerer bis...,abilify è usato per il trattamento degli episo...,abilify è impiegato per il trattamento di epis...
3,die injektionslösung wird zur schnellen kontro...,la soluzione iniettabile è usata per controlla...,la soluzione iniettabile è utilizzata per il c...
4,wie wird abilify angewendet?,Come si usa abilify?,come si usa abilify?


# **METRICS_COMPUTATION**

In [ ]:
my_data = pd.read_excel("german_italian_dt_ht.xlsx", index_col = 0)

In [ ]:
my_data

,original_sentences,direct_translation,reference_translation
0,wofür wird abilify angewendet?,A cosa serve Abilify,per che cosa si usa abilify?
1,es können auch depressive episoden auftreten.,possono verificarsi anche episodi di depressione.,possono inoltre verificarsi episodi di depress...
2,abilify wird zur behandlung mittelschwerer bis...,abilify è usato per il trattamento degli episo...,abilify è impiegato per il trattamento di epis...
3,die injektionslösung wird zur schnellen kontro...,la soluzione iniettabile è usata per controlla...,la soluzione iniettabile è utilizzata per il c...
4,wie wird abilify angewendet?,Come si usa abilify?,come si usa abilify?
5,bei schizophrenie beträgt die empfohlene anfan...,"in caso di schizofrenia, la dose iniziale racc...","in caso di schizofrenia, la dose di partenza r..."
6,die erhaltungsdosis beträgt 15 mg einmal tägli...,La dose di mantenimento è di 15 mg una volta a...,la dose di mantenimento è di 15 mg una volta a...
7,bei bipolarer störung beträgt die empfohlene a...,"In caso di disturbo bipolare, la dose iniziale...","per il disturbo bipolare, la dose di partenza ..."
8,bei einigen patienten kann auch eine höhere do...,In alcuni pazienti può essere utile un dosaggi...,alcuni pazienti possono trarre beneficio da un...
9,zur vorbeugung manischer episoden sollte die g...,La stessa dose deve essere somministrata per p...,"per prevenire episodi maniacali, la stessa dos..."


In [ ]:
my_data = pd.read_excel("german_italian_dt_ht.xlsx", index_col = 0)

direct_vs_human_translation_bleu = [[[sent1], sent2] for sent1, sent2 in zip(my_data["reference_translation"], my_data["direct_translation"] )]

direct_vs_human_translation_cosim = [[sent1, sent2] for sent1, sent2 in zip(my_data["reference_translation"], my_data["direct_translation"] )]

bleu_direct =  compute_scar_score(direct_vs_human_translation_bleu)[1]
cosim_direct = compute_similarity(direct_vs_human_translation_cosim)[1]

In [ ]:
bleu_direct =  compute_scar_score(direct_vs_human_translation_bleu)[1]
cosim_direct = compute_similarity(direct_vs_human_translation_cosim)[1]

In [ ]:
lexsem = []

for bleu, cosim in zip(bleu_direct, cosim_direct):
  lexsem.append(round((bleu + cosim)/2, 2))

In [ ]:
lexsem

[0.62,
 0.94,
 0.94,
 0.8,
 0.97,
 0.9,
 0.88,
 0.87,
 0.79,
 0.86,
 0.78,
 0.84,
 0.7,
 0.67,
 0.86,
 0.76,
 0.94,
 0.8,
 0.84,
 0.96,
 0.92,
 0.99,
 0.86,
 0.82,
 0.89,
 0.8,
 0.81,
 0.82,
 0.88,
 0.8,
 0.83,
 0.78,
 0.78]

In [ ]:
pd.DataFrame({"original_sentences":  my_data["original_sentences"] , "direct_translation": my_data["direct_translation"], "reference_translation": my_data["reference_translation"], "BLEU": bleu_direct, "Cosine Similarity": cosim_direct, "LexSem": lexsem}).to_csv("german_italian_translation_metrics.csv")

In [ ]:
pd.read_csv("german_italian_translation_metrics.csv", index_col=0)

,original_sentences,direct_translation,reference_translation,BLEU,Cosine Similarity,LexSem
0,wofür wird abilify angewendet?,A cosa serve Abilify,per che cosa si usa abilify?,0.51,0.74,0.62
1,es können auch depressive episoden auftreten.,possono verificarsi anche episodi di depressione.,possono inoltre verificarsi episodi di depress...,0.88,0.99,0.94
2,abilify wird zur behandlung mittelschwerer bis...,abilify è usato per il trattamento degli episo...,abilify è impiegato per il trattamento di epis...,0.92,0.96,0.94
3,die injektionslösung wird zur schnellen kontro...,la soluzione iniettabile è usata per controlla...,la soluzione iniettabile è utilizzata per il c...,0.71,0.90,0.80
4,wie wird abilify angewendet?,Come si usa abilify?,come si usa abilify?,0.95,1.00,0.97
5,bei schizophrenie beträgt die empfohlene anfan...,"in caso di schizofrenia, la dose iniziale racc...","in caso di schizofrenia, la dose di partenza r...",0.84,0.95,0.90
6,die erhaltungsdosis beträgt 15 mg einmal tägli...,La dose di mantenimento è di 15 mg una volta a...,la dose di mantenimento è di 15 mg una volta a...,0.77,0.99,0.88
7,bei bipolarer störung beträgt die empfohlene a...,"In caso di disturbo bipolare, la dose iniziale...","per il disturbo bipolare, la dose di partenza ...",0.77,0.97,0.87
8,bei einigen patienten kann auch eine höhere do...,In alcuni pazienti può essere utile un dosaggi...,alcuni pazienti possono trarre beneficio da un...,0.63,0.95,0.79
9,zur vorbeugung manischer episoden sollte die g...,La stessa dose deve essere somministrata per p...,"per prevenire episodi maniacali, la stessa dos...",0.78,0.95,0.86


In [ ]:
len(bleu_direct) == len(pd.read_csv("german_italian_translation_metrics.csv", index_col=0)["original_sentences"])

True

# **CORRELATION WITH HUMAN JUDGMENT**

In [ ]:
os.chdir("/content/drive/MyDrive/tesi/chapters/Lexsem validation/")


**CORRELATION WITH MEAN HUMAN JUDJMENT**

In [ ]:
my_data = pd.read_csv("german_italian_translation_metrics.csv", index_col=0)
my_data.head()

,original_sentences,direct_translation,reference_translation,BLEU,Cosine Similarity,LexSem
0,wofür wird abilify angewendet?,A cosa serve Abilify,per che cosa si usa abilify?,0.51,0.74,0.62
1,es können auch depressive episoden auftreten.,possono verificarsi anche episodi di depressione.,possono inoltre verificarsi episodi di depress...,0.88,0.99,0.94
2,abilify wird zur behandlung mittelschwerer bis...,abilify è usato per il trattamento degli episo...,abilify è impiegato per il trattamento di epis...,0.92,0.96,0.94
3,die injektionslösung wird zur schnellen kontro...,la soluzione iniettabile è usata per controlla...,la soluzione iniettabile è utilizzata per il c...,0.71,0.90,0.80
4,wie wird abilify angewendet?,Come si usa abilify?,come si usa abilify?,0.95,1.00,0.97


In [ ]:
dir_ = "/content/drive/MyDrive/tesi/chapters/Lexsem validation/italian-h-j"


In [ ]:
import glob
data_ = [file_ for file_ in glob.iglob(f'{dir_}/*.xlsx')]
data_

['/content/drive/MyDrive/tesi/chapters/Lexsem validation/italian-h-j/it-it-Antonio.xlsx',
 '/content/drive/MyDrive/tesi/chapters/Lexsem validation/italian-h-j/it-it-Beatrice.xlsx',
 '/content/drive/MyDrive/tesi/chapters/Lexsem validation/italian-h-j/it-it-SaraP.xlsx',
 '/content/drive/MyDrive/tesi/chapters/Lexsem validation/italian-h-j/it-it-Laura.xlsx',
 '/content/drive/MyDrive/tesi/chapters/Lexsem validation/italian-h-j/it-it-Lisa.xlsx',
 '/content/drive/MyDrive/tesi/chapters/Lexsem validation/italian-h-j/it-it-Stani.xlsx',
 '/content/drive/MyDrive/tesi/chapters/Lexsem validation/italian-h-j/it-it-Sophie.xlsx',
 '/content/drive/MyDrive/tesi/chapters/Lexsem validation/italian-h-j/it-it-Domenico.xlsx',
 '/content/drive/MyDrive/tesi/chapters/Lexsem validation/italian-h-j/it-it-Salvo.xlsx']

In [ ]:

human_jmnt = [pd.read_excel(file_, index_col=0)["judgment"].to_list() for file_ in data_]



In [ ]:

sum_human_jmnt = list(map(sum, zip(*human_jmnt)))

mean_human_jmnt = [round(x/9, 2) for x in sum_human_jmnt]



In [ ]:
df = pd.read_csv("german_italian_translation_metrics.csv", index_col=0)
import plotly.figure_factory as ff
import numpy as np

# Add histogram data
x2 = df["Cosine Similarity"]
x1 = df["BLEU"]
x3 = df["LexSem"]
x4 = mean_human_jmnt

# Group data together
hist_data = [x1, x2, x3, x4]

group_labels = [ 'BLEU SCORES','COSINE SIMILARITY SCORES', 'LexSem SCORES', "MEAN HUMAN JUDGMENT"]

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels, bin_size=[.0, .0, .0, .0])
fig.update_layout(title_text='METRICS DISTRIBUTION')
fig.show()

In [ ]:
from scipy.stats import spearmanr

s = spearmanr([x for x in my_data["LexSem"] ], mean_human_jmnt, axis=None)
print(s)

SignificanceResult(statistic=0.46949933263712373, pvalue=0.005841768296932882)


In [ ]:
from scipy.stats import spearmanr

s = spearmanr([x for x in my_data["BLEU"] ], mean_human_jmnt, axis=None)
print(s)

SignificanceResult(statistic=0.4876048611303821, pvalue=0.003998096013959063)


In [ ]:
from scipy.stats import spearmanr

s = spearmanr([x for x in my_data["Cosine Similarity"] ], mean_human_jmnt, axis=None)
print(s)

SignificanceResult(statistic=0.42492839646055125, pvalue=0.01370143961650437)


**Convertion to [0, 1, 2]**

In [ ]:
import glob
data_ = [file_ for file_ in glob.iglob(f'{dir_}/*.xlsx')]
data_

['/content/drive/MyDrive/tesi/chapters/Lexsem validation/italian-h-j/it-it-Antonio.xlsx',
 '/content/drive/MyDrive/tesi/chapters/Lexsem validation/italian-h-j/it-it-Beatrice.xlsx',
 '/content/drive/MyDrive/tesi/chapters/Lexsem validation/italian-h-j/it-it-SaraP.xlsx',
 '/content/drive/MyDrive/tesi/chapters/Lexsem validation/italian-h-j/it-it-Laura.xlsx',
 '/content/drive/MyDrive/tesi/chapters/Lexsem validation/italian-h-j/it-it-Lisa.xlsx',
 '/content/drive/MyDrive/tesi/chapters/Lexsem validation/italian-h-j/it-it-Stani.xlsx',
 '/content/drive/MyDrive/tesi/chapters/Lexsem validation/italian-h-j/it-it-Sophie.xlsx',
 '/content/drive/MyDrive/tesi/chapters/Lexsem validation/italian-h-j/it-it-Domenico.xlsx',
 '/content/drive/MyDrive/tesi/chapters/Lexsem validation/italian-h-j/it-it-Salvo.xlsx']

In [ ]:
def convert(r_data):
  converted = []
  human_j = pd.read_excel(r_data, index_col = 0)["judgment"].to_list()
  for x in human_j:
    if x <= 0.45:
      x = 0
    elif 0.46 < x <= 0.75:
      x = 1
    else:
      x = 2
    converted.append(x)
  
  return converted


   

In [ ]:
hmn_jmnt = [convert(x) for x in data_]
hmn_jmnt

[[2,
  1,
  2,
  2,
  2,
  0,
  2,
  0,
  2,
  2,
  1,
  0,
  2,
  1,
  1,
  2,
  2,
  1,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  1,
  2,
  1,
  2,
  0],
 [1,
  2,
  2,
  0,
  2,
  2,
  2,
  1,
  2,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  2,
  1,
  1,
  1,
  1,
  2,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0],
 [2,
  2,
  2,
  1,
  2,
  2,
  2,
  2,
  2,
  2,
  1,
  2,
  2,
  1,
  2,
  2,
  2,
  1,
  1,
  2,
  2,
  2,
  1,
  2,
  1,
  1,
  1,
  2,
  0,
  1,
  1,
  1,
  0],
 [2,
  2,
  2,
  2,
  2,
  1,
  2,
  1,
  2,
  2,
  1,
  1,
  2,
  1,
  1,
  2,
  2,
  1,
  2,
  2,
  2,
  2,
  2,
  1,
  1,
  1,
  2,
  2,
  2,
  1,
  1,
  1,
  1],
 [1,
  2,
  2,
  0,
  2,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  2,
  2,
  0,
  1,
  1,
  0,
  2,
  0,
  2,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0],
 [2,
  2,
  2,
  1,
  2,
  2,
  2,
  1,
  2,
  2,
  0,
  1,
  2,
  1,
  2,
  2,
  2,
  1,
  1,
  1,
  1,
  2,
  1,
  1,
  0,
  2,
  2,
  1,
  0,
  1,
  0,
  0,
  0],
 [1,

In [ ]:
sum_human_jmnt = list(map(sum, zip(*hmn_jmnt)))

human_jmnt = [round(x/9, 0) for x in sum_human_jmnt]



In [ ]:
human_jmnt 

[2.0,
 2.0,
 2.0,
 1.0,
 2.0,
 1.0,
 2.0,
 1.0,
 2.0,
 2.0,
 0.0,
 1.0,
 2.0,
 1.0,
 1.0,
 1.0,
 2.0,
 1.0,
 1.0,
 2.0,
 2.0,
 2.0,
 1.0,
 2.0,
 1.0,
 1.0,
 1.0,
 2.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0]

In [ ]:
pd.DataFrame({"original_sentences":  my_data["original_sentences"] , "direct_translation": my_data["direct_translation"], "reference_translation": my_data["reference_translation"], "BLEU": my_data["BLEU"], "Cosine Similarity": my_data["Cosine Similarity"], "LexSem": my_data["LexSem"], "human_judgment" : mean_human_jmnt }).to_csv("german_italian_final_metrics.csv")

In [ ]:
from scipy.stats import spearmanr

s = spearmanr([x for x in my_data["LexSem"] ], human_jmnt, axis=None)
print(s)

SignificanceResult(statistic=0.45304431380082216, pvalue=0.008106480272871693)


In [ ]:
from scipy.stats import spearmanr

s = spearmanr([x for x in my_data["Cosine Similarity"] ], human_jmnt, axis=None)
print(s)

SignificanceResult(statistic=0.4056339784497128, pvalue=0.01917908224975208)


In [ ]:
from scipy.stats import spearmanr

s = spearmanr([x for x in my_data["BLEU"] ], human_jmnt, axis=None)
print(s)

SignificanceResult(statistic=0.4705129020168959, pvalue=0.00572208536712898)


**SAVE ALL METRICS**

In [ ]:
pd.DataFrame({"original_sentences":  my_data["original_sentences"] , "direct_translation": my_data["direct_translation"], "reference_translation": my_data["reference_translation"], "BLEU": my_data["BLEU"], "Cosine Similarity": my_data["Cosine Similarity"], "LexSem": my_data["LexSem"], "human_judgment" : human_jmnt }).to_csv("german_italian_final_metrics.csv")

In [ ]:
df = pd.read_csv("german_italian_final_metrics.csv", index_col = 0)

In [ ]:
import plotly.express as px
fig = px.line(df , y=["LexSem", "human_judgment"],  markers=True, title = "LINE PLOT" )

fig.show()